
学习来自 
- https://www.jianshu.com/p/2fbf79c1de40
- http://www.voidcn.com/article/p-hwafvevv-bbk.html

##  1 特征选择


1、特征差异性。也就是一列特征值若几乎没有什么变化，那么这个特征对模型并没有什么作用。
2、相关性大小。也就是该维特征与target的相互作用有多大。

## 2 过滤法

- 1、方差判别
给定阈值，排除那些方差低于阈值的特征


```python
from sklearn.feature_selection import VarianceThreshold

#方差选择法，返回值为特征选择后的数据
#参数threshold为方差的阈值
VarianceThreshold(threshold=3).fit_transform(data.data)
```

- 2、相关性系数选择
计算各个特征对target的相关系数，常用的就是皮尔逊相关系数P来做衡量，给定我们想筛选出来的特征个数即可。

```python
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr

#选择K个最好的特征，返回选择特征后的数据
#第一个参数为计算评估特征是否好的函数，该函数输入特征矩阵和目标向量，输出二元组（评分，P值）的数组，数组第i项为第i个特征的评分和P值。在此定义为计算相关系数
#参数k为选择的特征个数
SelectKBest(lambda X, Y: array(map(lambda x:pearsonr(x, Y), X.T)).T, k=2).fit_transform(data.data, data.target)

```

- 3、卡方检验
卡方检验是一种用途很广的计数资料的假设检验方法。它属于非参数检验的范畴，主要是比较两个及两个以上样本率( 构成比）以及两个分类变量的关联性分析。其根本思想就是在于比较理论频数和实际频数的吻合程度或拟合优度问题。
换句话说，卡方检验是检验类别自变量对类别因变量的相关性。假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距，构建统计量：

$$
\chi^{2}=\sum \frac{(A-E)^{2}}{E}
$$                  

```python
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#选择K个最好的特征，返回选择特征后的数据
SelectKBest(chi2, k=2).fit_transform(data.data, data.target)
```

4、互信息法
互信息法也是用来评定类别自变量对类别因变量的相关性的。公式如下：

$$
I(X ; Y)=\sum_{x \in X} \sum_{y \in Y} p(x, y) \log \frac{p(x, y)}{p(x) p(y)}
$$

但是对于数值型变量，最大信息系数法被提出。

```
from numpy.ma import array
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import metrics as mr
from minepy import MINE
import numpy as np

X, y = load_iris(return_X_y=True)

# 自定义SelectKBest函数
def scores(X, y):
    mine = MINE()
    mic_scores = []
    for i in range(X.shape[1]):
      mine.compute_score(X[:, i], y)
      m = mine.mic()
      mic_scores.append(m)

    return np.array(mic_scores)

#选择K个最好的特征，返回特征选择后的数据
result = SelectKBest(scores, k=2).fit_transform(X, y)

```

## 3 包装消除特征

1、递归特征消除法
递归消除特征法使用一个基模型来进行多轮训练，每轮训练后，消除若干权值系数的特征，再基于新的特征集进行下一轮训练。使用feature_selection库的RFE类来选择特征的代码如下：

```python
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
 
#递归特征消除法，返回特征选择后的数据
#参数estimator为基模型
#参数n_features_to_select为选择的特征个数
RFE(estimator=LogisticRegression(), n_features_to_select=2).fit_transform(data.data, data.target)

```


## 4 集成法
集成法是用已有的机器学习模型对特征进行训练，得到各个特征的权值后进行筛选。例如，xgb的过程就是不断迭代对特征重要性进行排序以及收敛的过程，可以手动去做CV画出出各个特征的重要性排序进行比对，再对其进行筛选，而sklearn中的工具可以方便帮我们完成这项工作。
1、L1惩罚选择法
使用带惩罚项的基模型，除了筛选出特征外，同时也进行了降维。使用feature_selection库的SelectFromModel类结合带L1惩罚项的逻辑回归模型，来选择特征的代码如下：

```python
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
 
#带L1惩罚项的逻辑回归作为基模型的特征选择
SelectFromModel(LogisticRegression(penalty="l1", C=0.1)).fit_transform(data.data, data.target)
    
```

L1惩罚项降维的原理在于保留多个对目标值具有同等相关性的特征中的一个，所以没选到的特征不代表不重要。故，可结合L2惩罚项来优化。具体操作为：若一个特征在L1中的权值为1，选择在L2中权值差别不大且在L1中权值为0的特征构成同类集合，将这一集合中的特征平分L1中的权值，故需要构建一个新的逻辑回归模型：

```
from sklearn.feature_selection import SelectFromModel
 
#带L1和L2惩罚项的逻辑回归作为基模型的特征选择
#参数threshold为权值系数之差的阈值
SelectFromModel(LR(threshold=0.5, C=0.1)).fit_transform(data.data, data.target)

```
2、基于GDBT的特征选择

```
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier

#GBDT作为基模型的特征选择
SelectFromModel(GradientBoostingClassifier()).fit_transform(data.data, data.target)
```